In [ ]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as nd
import hyperspy.api as hs
import os
from orix.io import load
from orix.vector import Vector3d
from orix import plot
from tqdm import tqdm

In [ ]:
xmap = load("/Users/anders/Library/CloudStorage/OneDrive-NTNU/Prosjektoppgave/Data/"\
        "Processed/One_deg/error_fixed/ErMnO3_1_deg_scan_errors_fixed_rotated.hdf5")

folder = "/Users/anders/Library/CloudStorage/OneDrive-NTNU/Prosjektoppgave/Data/SPED"
file = "SPED_640x383x12_20x20_9p26x9p26nm_ScanRot20deg_CL12cm_spot0p5nm_alpha5"\
        "_NBD_10ms_1deg_fixed_scan.zspy"

dp = hs.load(os.path.join(folder, file), lazy = True)

In [ ]:
exclude_mask = xmap.correlation[:,0] < 0.001
exclude_mask = exclude_mask.reshape(dp.data.shape[:2])

include_mask = xmap.correlation[:,0] > 0.001
include_mask = include_mask.reshape(dp.data.shape[:2])

In [ ]:
#get a color ipf plot of Z projection

ckey = plot.IPFColorKeyTSL(xmap.phases["ErMnO3"].point_group, 
                                direction=Vector3d.zvector())

img_rgb = ckey.orientation2color(xmap.orientations)
img_rgb = img_rgb.reshape(dp.data.shape[:2] + (3,))

In [ ]:
#Get mask for each grain, using color
#Use binary dilation and erosion to fill holes in mask and remove boundaries

def get_mask(color, 
            threshold, 
            extra_erosion = False, 
            extra_dilation = False, 
            y_thresholding = None):
    
    mask = nd.binary_erosion(
            nd.binary_dilation(
               np.abs(img_rgb-np.asarray(color)).sum(axis=2)<threshold, iterations = 4), 
            iterations = 3,
            border_value =1 )
            
    mask = nd.binary_erosion(mask, iterations = 2)
    if extra_erosion:
        mask = nd.binary_erosion(mask, iterations = 3)

    if extra_dilation:
        mask = nd.binary_dilation(mask, iterations = 4)
        mask = nd.binary_fill_holes(mask)

    if y_thresholding is not None:
        mask[:y_thresholding,:] = 0
    
    return mask

In [ ]:
#define colors used for segmentation

grains = {"area1": [np.array([1,0.592,0.418]), 0.18],
"area2": [np.array([0.16,0.67,1]), 0.05],
"area3": [np.array([0.293,1,0.307]), 0.12],
"area4": [np.array([0.00668,0.0613,1]), 0.09],
"area5": [np.array([0.842,0.949,1]), 0.05],
"area6": [np.array([1,0.0487,0.0393]), 0.09, True],
"area7": [np.array([1,0.865,0.843]), 0.05, False, True],
"area8": [np.array([0.163,0.199,1]), 0.12, False , False, 50],
"area9": [np.array([0.338,1,0.884]), 0.13],
"area10": [np.array([0.506,1,0.374]), 0.13],

"area11": [np.array([1,0.25,0.374]), 0.13, True, False, 50],

"area12": [np.array([1,0.0226,0.353]), 0.13, True, True],

"area13": [np.array([1,0.432,0.0899]), 0.13, False, True],

"area14": [np.array([0.965,0.976,1]), 0.13, False, False, 100],

}


In [ ]:
#inspect that segmentaiton was successful

mask = np.zeros(img_rgb.shape[:2])
for i in range(14):
    area = f"area{i+1}"

    mask += get_mask(*grains[area]) * (i+1)

mask[mask>15] = 0
plt.figure()
plt.imshow(mask*include_mask,vmax = 15)

In [ ]:
#save segmented dataset for later use

for i in range(14):
    area = f"area{i+1}"

    area_mask = mask == i+1
    diff_img = (dp.data*area_mask).sum(axis = (2,3))
    np.save(f"diff_area{i+1}.npy", diff_img)

In [ ]:
#Define apertures for DF image of each segment

rois ={"area1" : hs.roi.CircleROI(cx=0.00339841, cy=-0.00563164, r=0.000825327, r_inner=0),
"area2": hs.roi.CircleROI(cx=-0.00883586, cy=-0.00388389, r=0.000825327, r_inner=0),
"area3" : hs.roi.CircleROI(cx=0.000194195, cy=0, r=0.000825327, r_inner=0),
"area4" : hs.roi.CircleROI(cx=-0.00116517, cy=-0.00388389, r=0.000825327, r_inner=0),
"area5" : hs.roi.CircleROI(cx=0.000194195, cy=-0.000194195, r=0.000825327, r_inner=0),
"area6" : hs.roi.CircleROI(cx=0.000291292, cy=-9.70973E-05, r=0.000825327, r_inner=0),
"area7" : hs.roi.CircleROI(cx=0.000194195, cy=-0.000388389, r=0.000825327, r_inner=0) ,
"area8" : hs.roi.CircleROI(cx=0.000194195, cy=-0.000291292, r=0.000825327, r_inner=0),
"area9" : hs.roi.CircleROI(cx=0.00951554, cy=-0.00903005, r=0.000825327, r_inner=0),
"area10" : hs.roi.CircleROI(cx=9.70973E-05, cy=-0.000291292, r=0.000825327, r_inner=0),
"area11" : hs.roi.CircleROI(cx=-0.00271872, cy=0.00301002, r=0.000825327, r_inner=0),
"area12": hs.roi.CircleROI(cx=0.000194195, cy=0, r=0.000825327, r_inner=0),
"area13": hs.roi.CircleROI(cx=0.000194195, cy=0, r=0.000825327, r_inner=0),
"area14" : hs.roi.CircleROI(cx=0.000679681, cy=0.00388389, r=0.000825327, r_inner=0)}


#define which areas are VBF images
VBF_image = [3,5,6,7,8,10,12,13]

#deine special brightness adjustments

brightness_adjustmets = {4: 2, 2:1/3, 11: 2.3/5, 14:2/3 }

In [ ]:
#create the combined DF image

img = np.zeros((383,640))

for i in tqdm(range(14)):
    area = f"area{i+1}"

    if i+1 in VBF_image: 
        scale = 1
    else:
        scale = 0.7
    
    roi = rois[area]
    temp_img = roi(dp.T).nansum().data.compute()

    temp_img /= (np.mean(temp_img)*scale)

    if i+1 in brightness_adjustmets.keys():
        temp_img *= brightness_adjustmets[i+1]

    mask = get_mask(*grains[area])
    img[mask] += temp_img[mask]

In [ ]:
#plot the result

domain_img = hs.signals.Signal2D(img * include_mask)

domain_img.axes_manager[0].offset = dp.axes_manager[0].offset
domain_img.axes_manager[1].offset = dp.axes_manager[1].offset
domain_img.axes_manager[0].scale = 6.92
domain_img.axes_manager[1].scale =6.92
domain_img.axes_manager[0].units = dp.axes_manager[0].units
domain_img.axes_manager[1].units = dp.axes_manager[1].units
domain_img.axes_manager[0].name = dp.axes_manager[0].name
domain_img.axes_manager[1].name = dp.axes_manager[1].name

domain_img.plot(title ="", vmin = 0, vmax = 6.232 , axes_off = True, colorbar = False)

plt.savefig("Combined_dark_field.svg")